In [193]:
# Import packages
import pandas as pd
import numpy as np
import re

In [194]:
# Read the CSV file
csv_file = 'Phase1 - Cleaned Data after using Excel.csv'
df = pd.read_csv(csv_file)

In [195]:
import re

# Define a function to extract details from the Vehicle_Title column
def extract_vehicle_details(title):
    # Check for specific models directly
    if "C-HR" in title:
        model = "C-HR"
    elif "Land Cruiser" in title:
        model = "Land Cruiser"
    else:
        # Use regex for other models if not matched by specific conditions
        model_match = re.search(r'\b(?:Toyota\s+[A-Za-z]+|86|RAV4|4Runner|MR2|T100|Previa|Avalon|Tacoma|Sienna|Highlander|ECHO|Prius|Corolla|Supra|Camry|FJ|Sequoia|Tercel|Yaris|Mirai|Venza|Paseo|Matrix|Tundra|Celica)\b', title, re.IGNORECASE)
        model = model_match.group() if model_match else None

    # Regular expressions for other components
    year_made = re.search(r'\b(19|20)\d{2}\b', title)
    body_type = re.search(r'\b(?:[A-Za-z]*cab)\b|\b(Minivan|Sedan|SUV|Hatchback|Truck|Coupe|Convertible|Van|Wagon|Pickup)\b', title, re.IGNORECASE)
    if body_type:
        body_type = "Cab" if body_type.group().lower().endswith("cab") else body_type.group()

    # Removed the version extraction

    doors = re.search(r'\b(\d)dr\b', title)
    engine = re.search(r'\((.*?)\)', title)  # Capture content within parentheses

    # Extract values or set as None if not found
    details = {
        "year_made": year_made.group() if year_made else None,
        "Model": model,
        "body_type": body_type if body_type else None,
        "doors": doors.group(1) if doors else None,
        "engine": engine.group() if engine else None
    }
    return details

# Apply the function to extract details from each row in Vehicle_Title
vehicle_details = df['Vehicle_Title'].apply(extract_vehicle_details)
vehicle_details_df = pd.DataFrame(vehicle_details.tolist())

# Concatenate with original data to check the results
extracted_data = pd.concat([df, vehicle_details_df], axis=1)


In [189]:
# Remove the word 'Toyota' from the 'Model' column if present
extracted_data['Model'] = extracted_data['Model'].str.replace(r'\bToyota\b\s*', '', regex=True)

In [190]:
#Quickly checking distinct values in a column
unique_values = extracted_data['Model'].unique()
print(unique_values)

['Previa' 'Avalon' 'Tacoma' 'RAV4' 'Sienna' '86' 'Highlander' 'ECHO'
 'Prius' 'Corolla' 'Supra' 'Camry' 'FJ' 'Sequoia' 'C-HR' 'Land Cruiser'
 'Tercel' '4Runner' 'Yaris' 'Mirai' 'Venza' 'Paseo' 'Matrix' 'MR2' 'T100'
 'Tundra' 'Celica']


In [191]:
# Extract car version details
def extract_version_if_else(title):
    # Define possible body types and door patterns
    body_types = ["Minivan", "Sedan", "SUV", "Hatchback", "Truck", "Coupe", "Convertible", "Van", "Wagon", "Pickup","Access Cab","Double Cab","Regular Cab","Extended Cab","Cab"]
    door_patterns = ["2dr", "3dr", "4dr"]

    # Initialize placeholders
    body_type_pos = None
    door_pos = None
    version = None

    # Find the first occurrence of any body type
    for body_type in body_types:
        if body_type in title:
            body_type_pos = title.find(body_type) + len(body_type)
            break

    # Find the first occurrence of any door pattern
    for door in door_patterns:
        if door in title:
            door_pos = title.find(door)
            break

    # If both positions are found, extract the text in between
    if body_type_pos is not None and door_pos is not None and body_type_pos < door_pos:
        version = title[body_type_pos:door_pos].strip()

    return version

# Apply the function to create a new 'Version' column in the DataFrame
extracted_data['Version'] = extracted_data['Vehicle_Title'].apply(extract_version_if_else)



In [192]:
#Saving the dataset
extracted_data.to_csv('Transformed_data.csv', index=False)